In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(2465)

# reading the file
import PyPDF2

text = ""

# Open the PDF file
with open('chatbot_data.pdf', 'rb') as file:
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(file)
    
    # Get the number of pages in the PDF
    num_pages = len(pdf_reader.pages)
    
    # Loop through all the pages and extract text
    for page_num in range(num_pages):
        # Get a page object
        page = pdf_reader.pages[page_num]
        
        # Extract text from the page
        page_text = page.extract_text()

        text = text + page_text

print(text)


# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)    

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()



Offer Letter: Graphic Designer  
Offer Letter  
Date:  June 22, 2024  
Employee:  Jessica Brown  
Address:  123 Creative Lane, Apt 4B, DesignTown, USA  
Email:  jessica.brown@example.com  
Phone:  +1-555-789-4561  
Employer:  Creative Agency Inc.  
Address:  456 Design Blvd, Suite 500, ArtCity, USA  
Email:  hr@creativeagency.com  
Phone:  +1-555-123-0987  
Position Offered:  Graphic Designer  
Department:  Creative  
Start Date:  July 15, 2024  
Salary:  $65,000 annually  
 
Job Description:  As a Graphic Designer at Creative Agency Inc., you will be responsible for 
creating visual content for various digital and print media projects.  
Benefits:  
• Health and dental insurance  
• Flexible working hours  
• Professional development programs  
 
Sample Questions and Answers:  
1. Question:  Who is the Employer in this Offer Letter?  
o Answer:  The Employer is Creative Agency Inc.  
2. Question:  What position is being offered to Jessica Brown?  
o Answer:  The position offered is Gr

In [2]:
for iter in range(5000):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == 4999:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 0.7499, val loss 1.0073
step 100: train loss 0.7363, val loss 0.9979
step 200: train loss 0.7463, val loss 1.0137
step 300: train loss 0.7353, val loss 1.0200
step 400: train loss 0.7354, val loss 0.9981
step 500: train loss 0.7283, val loss 1.0019
step 600: train loss 0.7272, val loss 0.9982
step 700: train loss 0.7046, val loss 0.9911
step 800: train loss 0.7283, val loss 0.9722
step 900: train loss 0.6903, val loss 0.9787
step 1000: train loss 0.7031, val loss 0.9666
step 1100: train loss 0.6997, val loss 0.9925
step 1200: train loss 0.6954, val loss 0.9512
step 1300: train loss 0.7064, val loss 0.9680
step 1400: train loss 0.6779, val loss 0.9610
step 1500: train loss 0.6959, val loss 0.9444
step 1600: train loss 0.6849, val loss 0.9486
step 1700: train loss 0.6830, val loss 0.9491
step 1800: train loss 0.6883, val loss 0.9203
step 1900: train loss 0.6898, val loss 0.9417
step 2000: train loss 0.6740, val loss 0.9320
step 2100: train loss 0.6727, val loss 0.9378


In [27]:
def uncertainty_sampling(model, data, n_samples):
    model.eval()
    uncertainties = []

    for i in range(0, len(data) - block_size, block_size):
        x = data[i:i+block_size].unsqueeze(0).to(device)
        with torch.no_grad():
            logits, _ = model(x)
        probs = F.softmax(logits, dim=-1)
        uncertainty = -torch.max(probs, dim=-1)[0].mean().item()
        uncertainties.append((uncertainty, i))

    uncertainties.sort(reverse=True, key=lambda x: x[0])
    selected_indices = [idx for _, idx in uncertainties[:n_samples]]
    
    model.train()
    return selected_indices


def label_samples(data, indices):
    x = torch.stack([data[i:i+block_size] for i in indices])
    y = torch.stack([data[i+1:i+block_size+1] for i in indices])
    return x, y


In [ ]:
# Number of new samples to query in each active learning iteration
n_new_samples = 100

for iter in range(max_iters):
    # Evaluate the loss on train and val sets periodically
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    # Perform active learning every eval_interval iterations
    if iter % eval_interval == 0 and iter > 0:
        new_sample_indices = uncertainty_sampling(model, train_data, n_new_samples)
        new_x, new_y = label_samples(train_data, new_sample_indices)
        new_x, new_y = new_x.to(device), new_y.to(device)
        
        # Train on the newly labeled samples
        logits, loss = model(new_x, new_y)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    # Sample a batch of data
    xb, yb = get_batch('train')

    # Evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


In [9]:
idx = torch.tensor(encode('What is probationary period?')).to(device=device)
idx = idx.unsqueeze(0)
idx

tensor([[48, 60, 53, 72,  1, 61, 71,  1, 68, 70, 67, 54, 53, 72, 61, 67, 66, 53,
         70, 77,  1, 68, 57, 70, 61, 67, 56, 24]], device='cuda:0')

In [10]:
def generate(model, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = model(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [11]:
print(decode(generate(m,idx, max_new_tokens=200)[0].tolist()))

What is probationary period?  
o Answer:  The annual salary identifications?  
o Answer:  Yes, Daniel?  
• Answer:  The offer?  
o Answer:  Marketing Moelary, Suite 401, Han Semite Eth. Acalers  
2. Emily Parkel ware.kellori@exa


In [50]:
torch.save(m, 'contract_analysis_model.pth')


In [49]:
torch.save(m.state_dict(), 'contract_analysis_chatbot.pth')
